In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

AWS_ACCESS_KEY = os.getenv("ACCESS_KEY")
AWS_SECRET_KEY = os.getenv("SECRET_KEY")

WEAVIATE_CLOUD_URL = os.getenv("WEAVIATE_CLOUD_URL")
WEAVIATE_CLOUD_KEY = os.getenv("WEAVIATE_CLOUD_KEY_READ")

## Connect

In [ ]:
import weaviate, json

# Connect to a Weaviate Cloud instance
client = weaviate.connect_to_wcs(
    cluster_url=WEAVIATE_CLOUD_URL,
    auth_credentials=weaviate.auth.AuthApiKey(WEAVIATE_CLOUD_KEY),

    headers={
        "X-AWS-Access-Key": AWS_ACCESS_KEY,
        "X-AWS-Secret-Key": AWS_SECRET_KEY,
    },


    # Set longer timeout
    additional_config=weaviate.config.AdditionalConfig(timeout=(500, 1500))
)

# Connect to the local instance deployed with Docker Compose
# client = weaviate.connect_to_local(
#     headers={
#         "X-AWS-Access-Key": AWS_ACCESS_KEY,
#         "X-AWS-Secret-Key": AWS_SECRET_KEY,
#     },
#
#     # Set longer timeout
#     additional_config=weaviate.config.AdditionalConfig(timeout=(500, 1500))
# )

client.is_ready()

## RAG - single prompt

In [ ]:
papers = client.collections.get("Papers")

response = papers.query.near_text("challenges of zero shot training", limit=5)

for item in response.objects:
    print(item.properties["chunk"])

In [ ]:
papers = client.collections.get("Papers")

response = papers.generate.near_text(
    "challenges of zero shot training",
    limit=5,
    single_prompt="What are the challenges of zero shot training? Please answer based on the following chunk: {chunk}"
)

for item in response.objects:
    print(item.properties["chunk"])
    print(item.generated)
    print("--------------------\n")

In [ ]:
papers = client.collections.get("Papers")

response = papers.generate.near_text(
    "zero shot training",
    limit=5,
    single_prompt="Please explain what the following chunk of text talks about in the context of zero-shot training: {chunk}"
)

for item in response.objects:
    print(item.properties["chunk"])
    print(item.generated)
    print("----------------------------------------\n")

## RAG - group task

In [ ]:
papers = client.collections.get("Papers")

response = papers.generate.near_text(
    "difficulties and challenges of zero shot training",
    limit=5,
    grouped_task="What can you tell me about zero shot training? Please answer based on the provided content",
)

for item in response.objects:
    print(item.properties["chunk"])
    
print("----------------------------------------\n")
print(response.generated)

In [ ]:
papers = client.collections.get("Papers")

response = papers.generate.near_text(
    "challenges of zero shot training",
    limit=5,
    grouped_task="What can you tell me about zero shot training? Please answer based on the provided content",
    grouped_properties=["chunk", "title"]
)

for item in response.objects:
    print(item.properties["chunk"])
    
print("----------------------------------------")
print(response.generated)

## Close the client when done

In [ ]:
client.close()